<a href="https://colab.research.google.com/github/akhileshmatta/Bigdata-Health-app/blob/master/Major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/Treatment

/content/drive/MyDrive/Treatment


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


dis_sym_dataset_comb.csv  dis_sym_dataset_norm.csv  __pycache__  Treatment.py


In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from time import time
from collections import Counter
import operator
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import math
from Treatment import diseaseDetail

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import GradientBoostingClassifier

warnings.simplefilter("ignore")

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

In [ ]:
df_comb = pd.read_csv("/content/drive/MyDrive/Treatment/dis_sym_dataset_comb.csv") # Disease combination
df_norm = pd.read_csv("/content/drive/MyDrive/Treatment/dis_sym_dataset_norm.csv") # Individual Disease

X = df_norm.iloc[:, 1:]
Y = df_norm.iloc[:, 0:1]

In [ ]:
dataset_symptoms = list(X.columns)

In [ ]:


X = df_comb.iloc[:, 1:]
Y = df_comb.iloc[:, 0:1]

In [ ]:
lr = LogisticRegression()
lr = lr.fit(X, Y)
lr.score(X, Y)
# scores = cross_val_score(lr, X, Y, cv=5)

0.9077532541029995

In [ ]:
user_symptoms = str(input("Please enter symptoms separated by comma(,):\n")).lower().split(',')
# Preprocessing the input symptoms
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)

Please enter symptoms separated by comma(,):
headache fever cough


In [ ]:
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
# query expansion performed by joining synonyms found for each symptoms initially entered
print("After query expansion done by using the symptoms entered")
print(user_symptoms)

After query expansion done by using the symptoms entered
['vexation coughing febricity worry fever cough cephalalgia headache fever cough febrility headache feverishness concern pyrexia head ache']


In [ ]:
# Loop over all the symptoms in dataset and check its similarity score to the synonym string of the user-input 
# symptoms. If similarity>0.5, add the symptom to the final list
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [ ]:
# Print all found symptoms
print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)
    
# Show the related symptoms found in the dataset and ask user to select among them
select_list = input("\nPlease select the relevant symptoms. Enter indices (separated-space):\n").split()

# Find other relevant symptoms from the dataset based on user symptoms based on the highest co-occurance with the
# ones that is input by the user
dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])

Top matching symptoms from your search!
0 : fever
1 : headache
2 : coughing

Please select the relevant symptoms. Enter indices (separated-space):
0


In [ ]:
# Symptoms that co-occur with the ones selected by user              
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   
#print(dict_symp_tup) 

In [ ]:
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = [] 


Common co-occuring symptoms:
0 : headache
1 : testicular pain
2 : vomiting
3 : barky cough
4 : sore throat
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
-1

Common co-occuring symptoms:
0 : maculopapular rash
1 : confusion
2 : diarrhea
3 : feeling tired
4 : swollen lymph node
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
-1

Common co-occuring symptoms:
0 : shortness breath
1 : unintended weight loss
2 : runny nose
3 : chest pain
4 : large lymph node
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
-1

Common co-occuring symptoms:
0 : muscle weakness
1 : tiredness
2 : seizure
3 : nausea
4 : joint bone pain
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
-2

Common co-occuring symptoms:
0 : non itchy skin ulcer
1 : vaginal bleeding
2 : muscle 

In [ ]:
print("\nFinal list of Symptoms that will be used for prediction:")
sample_x = [0 for x in range(0,len(dataset_symptoms))]
for val in final_symp:
    print(val)
    sample_x[dataset_symptoms.index(val)]=1


Final list of Symptoms that will be used for prediction:
fever
nausea
enlarged lymph node neck


In [ ]:
#KNN classifier

In [ ]:
clf = KNeighborsClassifier()
clf.fit(X, Y)

KNeighborsClassifier()

In [ ]:
clf.score(X, Y)

0.8805885681946802

In [ ]:
clf.predict([sample_x])

array(['Lymphoma'], dtype=object)

In [ ]:
#Nusvc classifier

In [ ]:
nscv=NuSVC()

In [ ]:
#Random Forest classifier

In [ ]:
clf = RandomForestClassifier()
clf.fit(X, Y)

RandomForestClassifier()

In [ ]:
clf.predict([sample_x])

array(['Hepatitis A'], dtype=object)

In [ ]:
#Naive Bayes classifier

In [ ]:
gnb = GaussianNB()
gnb.fit(X, Y)

GaussianNB()

In [ ]:
gnb.score(X, Y)

0.9130730050933786

In [ ]:
gnb.predict([sample_x])

array(['Mononucleosis'], dtype='<U60')

In [ ]:
#Xgboost

In [ ]:
Y[0:1].index

RangeIndex(start=0, stop=1, step=1)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
Y_i = le.fit_transform(Y)

xg = XGBClassifier()
xg.fit(X, Y_i)

In [ ]:
#Logistic Regression

In [ ]:
lr = LogisticRegression()
lr = lr.fit(X, Y)
prediction = lr.predict_proba([sample_x])

In [ ]:
k = 10
diseases = list(set(Y['label_dis']))
diseases.sort()
topk = prediction[0].argsort()[-k:][::-1]

In [ ]:
topk

array([189, 185, 159, 179, 217,  46,  44, 200,  33, 114])

In [ ]:
print(f"\nTop {k} diseases predicted based on symptoms")
topk_dict = {}
# Show top 10 highly probable disease to the user.
for idx,t in  enumerate(topk):
    match_sym=set()
    row = df_norm.loc[df_norm['label_dis'] == diseases[t]].values.tolist()
    row[0].pop(0)

    for idx,val in enumerate(row[0]):
        if val!=0:
            match_sym.add(dataset_symptoms[idx])
    prob = (len(match_sym.intersection(set(final_symp)))+1)/(len(set(final_symp))+1)
    prob *= mean(scores)
    topk_dict[t] = prob
j = 0
topk_index_mapping = {}
topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
for key in topk_sorted:
  prob = topk_sorted[key]*100
  print(str(j) + " Disease name:",diseases[key], "\tProbability:",str(round(prob, 2))+"%")
  topk_index_mapping[j] = key
  j += 1

select = input("\nMore details about the disease? Enter index of disease or '-1' to discontinue and close the system:\n")
if select!='-1':
    dis=diseases[topk_index_mapping[int(select)]]
    print()
    print(diseaseDetail(dis))


Top 10 diseases predicted based on symptoms
0 Disease name: Postpartum depression/ Perinatal depression 	Probability: 89.19%
1 Disease name: Polycystic ovary syndrome (PCOS) 	Probability: 29.73%
2 Disease name: Myocardial Infarction (Heart Attack) 	Probability: 29.73%
3 Disease name: Perennial Allergic Conjunctivitis 	Probability: 29.73%
4 Disease name: Scrapie 	Probability: 29.73%
5 Disease name: Celiacs disease 	Probability: 29.73%
6 Disease name: Carpal Tunnel Syndrome 	Probability: 29.73%
7 Disease name: Rabies 	Probability: 29.73%
8 Disease name: Breast Cancer / Carcinoma 	Probability: 29.73%
9 Disease name: Hyperthyroidism 	Probability: 29.73%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
0

Postpartum depression/ Perinatal depression
Other names -  Postnatal depression 
Specialty -  Psychiatry 
Symptoms -  Extreme sadness, low energy, anxiety, changes in sleeping or eating patterns, crying episodes, irritability   
Usual on